In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from load.info import Info

In [2]:
wdir = "/home/hoseung/Work/data/"
info = Info(nout=187, base=wdir + '29172')
mh, mg = pickle.load(open(wdir + "matched_hal_gal.pickle", "rb"))
# mh and mg are mere HaloMaker catalogs

In [3]:
prg = pickle.load(open(wdir + "main_prgs_final_augmented_5_10_0.5_0.5_0.5_37_0.01_filtered_.pickle", "rb"))
prg = [gal for gal in prg if gal.data["mstar"][0] > 5e9]

In [4]:
ind = np.lexsort( (mg["id"], mg["host"]) ) 
sortd_mg = mg[ind]
sortd_mh = mh[ind]
# Note that the last key in lexsort is primary while the first key in sorted is primary.
sortd_prg = sorted(prg, key = lambda gal: (gal.cluster, gal.ids[0]))
# Check
#for i, gal in enumerate(sortd_prg[:10]):
#    print(gal.cluster, gal.ids[0], sortd_mg["host"][i], sortd_mg["id"][i])

In [8]:
# Rvir Vs 10*Reff
Reff = np.array([10 * min([40, gal.data["reff"][0]]) for gal in sortd_prg])
Mstar = np.array([gal.data["mstar"][0] for gal in sortd_prg])
fig, ax = plt.subplots()
fig.suptitle(r"$R_{vir} Vs R_{eff}$")
cb = ax.scatter(np.log10(sortd_mh["rvir"] * info.boxtokpc),
           np.log10(Reff),
           c=np.log10(Mstar), cmap="jet", edgecolor="none", alpha=0.8)
fig.colorbar(cb, label="log10(" + r"$M_{\ast}$)")
ax.plot([1,3.5], [1,3.5], "r:", lw=3)
ax.set_aspect("equal")
ax.set_xlabel("log10(" + r"R$_{vir}$ / kpc)")
ax.set_ylabel("log10(" + r"10R$_{eff}$ /kpc)")
#plt.show()
plt.savefig(wdir + "figs/Rvir_Reff.png")

# Moster plot

In [104]:
def get_M1(z, M10=11.59 , M11=1.195):
    return M10 + M11*(z/(z+1))

def m_M(M,z):
    """
    Moster SHM relation from Moster et al. 2013
    Returns M*/Mhalo.
    
    Parameters
    ----------
    M : float
        Halo Mass
    z : float
        Redshift
    """
    N = get_M1
    N10 = 0.0351
    N11 = -0.0247

    beta = get_M1
    beta10 = 1.376
    beta11 = -0.826
 
    gamma = get_M1
    gamma10 = 0.608
    gamma11 = 0.329

    M1 = 10**get_M1(z, 11.59, 1.195)
    nn = N(z, N10, N11)
    bb = beta(z, beta10, beta11)
    gg = gamma(z, gamma10, gamma11)

    return 2 * nn / ( (M/M1)**(-bb) + (M/M1)**(gg))

In [105]:
mm = np.logspace(10, 15, 100)
#plt.plot(np.log10(xp), np.log10(m_M(mm,0)*mm))

In [127]:
# Moster plot (Moster et al. 2013)
fig, ax = plt.subplots(1,2)
fig.set_size_inches(12,6)

#ax.plot([1,3.5], [1,3.5], "r:", lw=3)
i_central = sortd_mh["level"] == 1

ax[1].set_aspect("equal")
ax[1].set_xlabel("log10(" + r"M$_{vir, hal} / M_{\odot}$)" )
ax[1].set_ylabel("lgo10(" + r"M$_{\ast, gal} / M_{\odot}$)")

ax[0].scatter(np.log10(sortd_mh["mvir"][~i_central]),
              np.log10(Mstar[~i_central]), label="satellite",
              color="blue", edgecolor="none", alpha=0.3)
ax[0].scatter(np.log10(sortd_mh["mvir"][i_central]),
              np.log10(Mstar[i_central]), label="central",
              color="red", edgecolor="none", alpha=0.3)

ax[1].scatter(np.log10(sortd_mh["mvir"][~i_central]), 
              Mstar[~i_central] / sortd_mh["mvir"][~i_central] / (info.ob/info.om),
              label="satellite", color="blue", edgecolor="none", alpha=0.3)
ax[1].scatter(np.log10(sortd_mh["mvir"][i_central]), 
              Mstar[i_central] / sortd_mh["mvir"][i_central] / (info.ob/info.om),
              label="central", color="red", alpha=0.3)

ax[0].plot(np.log10(mm), np.log10(m_M(mm,0) * mm), label="Moster 13")
ax[1].plot(np.log10(mm), m_M(mm,0) / (info.ob/info.om), label="Moster 13")

ax[1].legend()
#ax[1].legend()

#ax[0].set_yscale("log")
ax[1].set_yscale("log")
#ax[0].set_ylim([0,5])
#ax[1].set_ylim([0,2])

ax[0].set_ylabel("log10(" + r"$M_{\star} / M_{\odot})$")
ax[0].set_xlabel("log10(" + r"$M_{200} / M_{\odot}$)")
ax[0].set_title("Stellar-Halo mass")

ax[1].set_ylabel(r"$ M_{\ast} / M_{b}$")
ax[1].set_xlabel("log10(" + r"$M_{200} / M_{\odot}$)")
ax[1].set_title("Baryon conversion effeciency")
plt.tight_layout()
plt.savefig("./ALL_Moster_plot.png")

/home/hoseung/mypy/lib/python3.5/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/hoseung/mypy/lib/python3.5/site-packages/matplotlib/axes/_base.py:1327: UserWarning: aspect is not supported for Axes with xscale=linear, yscale=log
  'yscale=%s' % (xscale, yscale))
